<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Цель" data-toc-modified-id="Цель-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><strong>Цель</strong></a></span></li><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><strong>Описание данных</strong></a></span></li><li><span><a href="#План-действий" data-toc-modified-id="План-действий-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>План действий</a></span></li></ul></li><li><span><a href="#Обзор-и-предобработка-данных" data-toc-modified-id="Обзор-и-предобработка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор и предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Построение-модели-линейной-регрессии" data-toc-modified-id="Построение-модели-линейной-регрессии-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Построение модели линейной регрессии</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Прибыль-и-риски-для-каждого-региона" data-toc-modified-id="Прибыль-и-риски-для-каждого-региона-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Прибыль и риски для каждого региона</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Построение регрессионной модели для определения региона максимальной прибыли с добычи нефти из новых скважин компании «ГлавРосГосНефть»

### **Цель**

Используя данные о скважинах из трёх регионов, построить модель линейной регрессии для предсказания суммарного объёма добычи нефти, прибыли и рисков в каждом из них и выбрать лучший (более прибыльный и менее рискованный).

### **Описание данных**

Работать будем с 3-мя датасетами - соответствуют трём регионам. Каждый состоит из:
- `id` — уникальный идентификатор скважины;  
- `f0`, `f1`, `f2` — три признака скважины (данные синтетические);  
- `product` — объём запасов в скважине (тыс. баррелей).  


### План действий

- импорт данных и необходимых библиотек
- обзор данных (пропуски, дубликаты, прочие аномалии, гистограммы и корреляции)
- предобработка данных
- построение модели линейной регрессии и обучение её отдельно на признаках каждого региона
- расчёт среднего запаса сырья в скважине и RMSE модели для каждого региона
- расчёт достаточности объёма сырья в скважине для безубыточности добычи
- методом Bootstrap с 1000 итераций случайно выбрать 500 скважин, из них выбрать 200 с максимальным предсказанным запасом сырья, чтобы по каждому региону получить:
  - распределение суммарной прибыли лучших предсказанных скважин
  - 95% доверительный интервал этой прибыли
  - риск убытков (отрицательной прибыли)
- выводы

## Обзор и предобработка данных

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import warnings 
warnings.filterwarnings('ignore')

from decimal import Decimal

In [ ]:
try:
    df0 = pd.read_csv('datasets/geo_data_0.csv')
    df1 = pd.read_csv('datasets/geo_data_1.csv')
    df2 = pd.read_csv('datasets/geo_data_2.csv')
except FileNotFoundError: 
    df0 = pd.read_csv('geo_data_0.csv')
    df1 = pd.read_csv('geo_data_1.csv')
    df2 = pd.read_csv('geo_data_2.csv')

In [ ]:
df_all = [df0, df1, df2]
i=0
for _ in df_all:
    display('*'*50)
    display(f'Регион {i}')
    display(_.head(2), '_ '*25)
    display(_.info(), '_ '*25)
    display('Дубликаты:', _.duplicated().sum(), '_ '*25)
    display('Пропуски:', _.isna().sum())
    i+=1

In [ ]:
#для стоящей перед нами задачи признак id скважины - лишний. удалим.
df0.drop('id', errors='ignore', inplace=True, axis=1)
df1.drop('id', errors='ignore', inplace=True, axis=1)
df2.drop('id', errors='ignore', inplace=True, axis=1)
df_all = [df0,df1,df2]

In [ ]:
#отобразим диаграммы рассеяния и посчитаем корреляцию Пирсона
i=0
for _ in df_all:
    display('*'*100, f'Регион {i}')
    px.scatter_matrix(_, opacity=.005, height=800).show()
    px.imshow(_.corr().round(3)*100, text_auto=True).show()
    i+=1

In [ ]:
#отобразим распределения признаков
i=0
for _ in df_all:
    display('*'*100, f'Регион {i}')
    i+=1
    for column in _.columns:
        display('_ '*50)
        fig = px.histogram(_, column)
        fig.show()

### Вывод

- 100тыс. записей (скважин) в каждом регионе, без дубликатов и пропусков
- признак `id` для наших целей лишний - удалили
- наблюдаем красивые зависимости одних признаков от других на диаграммах рассеяния, разные значения их корреляций по Пирсону, вплоть до 44%. По-хорошему перед построением модели линейной регрессии стоит отмасштабировать и отнормировать признаки, а также избавиться от мультиколлинеарности. Процедуру стандартизации данных проведём ниже, избавляться от мультиколлинеарности не будем (выходит за рамки проекта).

## Построение модели линейной регрессии

In [ ]:
#data = {'df0':[[features_train, target_train], [features_test, target_test]], 'df1':...}
#по каждому региону разобьём датасет на тренировочный и тестовый в соотношении 3:1
#отмасштабируем признаки методом StandardScaler()
#положим получившиеся данные в контейнер data
data = dict()
i=0
for _ in df_all:
    target = _['product']
    features = _.drop('product', errors='ignore', axis=1)
    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=.25, random_state=42)
    scaler = StandardScaler()
    scaler.fit(features_train)
    scaler.transform(features_train)
    scaler.transform(features_test)
    train = [features_train, target_train]
    test = [features_test, target_test]
    df = 'df'+ str(i)
    data[df]=[train, test]
    i+=1

In [ ]:
#чтобы удостовериться, что данные отмасштабированы, отобразим новые расределения признаков
for _ in ['df0', 'df1', 'df2']:
    display('*'*50, _)
    display('features_train')
    px.histogram(data[_][0][0], barmode='overlay').show()
    display('features_test')
    px.histogram(data[_][1][0], barmode='overlay').show()

In [ ]:
#построим модель линейной регрессии для каждого региона
#предскажем запасы скважин на тестовой выборке
#посчитаем RMSE
#barrels = {df0: [predicted, valid], df1:...}
#rmse = {df0: rmse0, df1:...}
barrels=dict()
rmse=dict()
for _ in data:
    model = LinearRegression()
    model.fit(data[_][0][0], data[_][0][1])
    barrels[_] = [model.predict(data[_][1][0]), data[_][1][1].to_numpy()]
    rmse[_] = mean_squared_error(barrels[_][1], barrels[_][0], squared=False).round(1)

In [ ]:
#средний запас сырья в скажине по региону, предсказанный моделью
[np.mean(barrels[_][0]).round(1) for _ in data]

In [ ]:
#RMSE предсказания
list(rmse.values())

### Вывод

- модель линейной регрессии для каждого региона построена
- предсказания для скважин из тестовой выборки получены и сохранены в контейнере `barrels`
- модель отрабатывает предельно точно в регионе 1 (df1), тогда как в 0-ом и 2-ом RMSE больше трети среднего запаса скважины, что увеличивает риски потерь.

## Прибыль и риски для каждого региона

In [ ]:
WELLS_PCS = 500 # столько скважин в новом регионе случайно отбирают для
#изучения перед принятием решения о начале разработки

BEST_WELLS_PCS = 200 # из случайно отобранных выбирают столько лучших по прогнозу модели
TRIES = 1000 # бутстрап: столько раз повторяют эскперимент по выбору лучших из изначальных
REVENUE_PER = 450e3 # доход с 1тыс.бар. в рублях
BUDGET = 10e9 # бюджет на разработку всех скважин (best_wells_pcs) в регионе

In [ ]:
#минимальный средний объем скважины в тыс.бар., чтобы не уйти в убыток от разработки в регионе
well_size_min = BUDGET / (REVENUE_PER * BEST_WELLS_PCS)
display(f'Мин. допустимый объём скважины, тыс.бар.: {round(well_size_min,1)}')

In [ ]:
#перенесём данные из контейнера barrels типа dict в контейнер с типом DataFrame, чтобы можно было удобно
#отсортировать скважины по убыванию запасов нефти в них
barrels_df = [
    pd.DataFrame(barrels[_], index=['predicted','valid']).T.sort_values('predicted', ascending=False)
    for _ in data]

In [ ]:
#шаг ради интереса
#отберём лучшие предсказанные скважины в каждом регионе, и для этой выборки посчитаем среднее и RMSE
display('mean', [barrels_df[_][:BEST_WELLS_PCS].predicted.mean().round(1) for _ in [0,1,2]])
display('RMSE', [mean_squared_error(barrels_df[_][:BEST_WELLS_PCS].predicted,
                            barrels_df[_][:BEST_WELLS_PCS].valid,
                            squared=False).round(1) 
         for _ in [0,1,2]])
display('revenue', ['%.2E' % Decimal(barrels_df[_][:BEST_WELLS_PCS].valid.sum()*REVENUE_PER-BUDGET) for _ in [0,1,2]])

#RMSE уменьшилось, средний объём скважины увеличился и стал больше найденного ранее минимального объёма,
#необходимого для безубыточной разработки в регионе. 
#Прибыль (доход с 200-от скважин минус разработка) положительны, больше 2.4млрдР.
#но этих данных пока недостаточно для принятия решения по региону, т.к. мы схитрили, взяв 200 лучших
#месторождений из всех-всех 25000, доступных в тестовых выборках. В реальности же у нас будет только 500
#случайных исследованных скважин, из которых будем отбирать лучшие. Поэтому нужен бустрап.

In [ ]:
#применим бутстрап и создадим датафрейм, где каждое значение - суммарная прибыль 200 лучших месторождений.
state = np.random.RandomState(42)
revenue_bootstrap = []
for _ in barrels_df:
    revenue_region = []
    for i in range(TRIES):
        revenue_region.append(_.sample(n=WELLS_PCS, replace=True, random_state=state)\
        .sort_values('predicted', ascending=False)[:BEST_WELLS_PCS].valid.sum()*REVENUE_PER-BUDGET)
    revenue_bootstrap.append(pd.Series(revenue_region))
revenue_bootstrap = pd.DataFrame(revenue_bootstrap).T

In [ ]:
px.box(revenue_bootstrap)
#пока регион 1 (df1) выглядит более многообещающим по сравнению с остальными

In [ ]:
display('Средняя прибыль', ['%.2E' % Decimal(revenue_bootstrap[_].mean()) for _ in revenue_bootstrap.columns])
display('95% доверительный интервал', 
        [['%.2E' % Decimal(revenue_bootstrap[_].quantile(.025)), '%.2E' % Decimal(revenue_bootstrap[_].quantile(.975))] 
         for _ in revenue_bootstrap.columns])
display('Риск убытков, %', [revenue_bootstrap[_].lt(0).mean()*100 for _ in revenue_bootstrap.columns])

### Вывод

Среди трёх регионов `df1` более выгоден: 
- медианное и среднее значение прибыли больше
- 95% доверительный интервал у́же и целиком находится в области положительных прибылей
- риск убытков минимален, в разы меньше риска в остальных регионах

## Вывод

__Регион 1 (df1)__ - самый обещающий: риск убытков __1.6%__, с вероятностью 95% прибыль находится в интервале __0.03-0.8 млрдР__ и в среднем составляет __0.44 млрдР__.  
Средний предсказанный запас скважины в этом регионе: __68.7тыс.бар.__ (в остальных регионах на треть больше).  RMSE: __0.9тыс.бар.__, модель предсказывает предельно точно (в остальных регионах ошибка в 40 раз больше)

- для нахождения лучшего региона были задействованы модель линейной регрессии и метод `bootstrap` (1000 раз случайно выбирали 500 скважин из 25000, а из низ выбирали 200 лучших по предсказанию модели)
- анализ основан на выборке в 100тыс. скважин для каждого региона
- данные на входе - чистые (без пропусков, аномалий, дубликатов, в нужном формате)
- признаки были стандартизованы методом `StandardScaler`
- в 0-ом регионе (df0) наблюдается слабая (44%) корреляция между признаками f0 и f1 - по-хорошему от неё нужно избавиться, это может улучшить прогноз модели  
- минимальный (для безубыточности) средний запас скважины (среди 200-от отобранных для разработки) составляет __111тыс.бар.__